In [11]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth('monitoring.evaluation.impactyouth@carisfoundationintl.org','P@$$w0rd!2#4')
pay ={"xmlns":"http://openrosa.org/formdesigner/DD8FE78F-C33C-4D7D-A63B-AEDFF3A85279","limit":1000}
forms = get('https://www.commcarehq.org/a/caris-test/api/v0.5/form/',auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"https://www.commcarehq.org/a/caris-test/api/v0.5/form/{__meta['next']}",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 1150}


1150

In [12]:
DataFrame(list(
    map(
        lambda k: {
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd']
        } 
        ,__objects
    )
))

,username,timeEnd
0,5djeff,2022-02-21T20:09:04.285000Z
1,5djeff,2022-02-21T20:08:34.829000Z
2,5djeff,2022-02-21T20:08:10.887000Z
3,5djeff,2022-02-21T20:07:31.388000Z
4,5djeff,2022-02-21T20:07:05.759000Z
...,...,...
1145,mcarthur,2022-02-16T20:26:50.145000Z
1146,mcarthur,2022-02-16T20:02:48.005000Z
1147,mcarthur,2022-02-16T20:00:56.130000Z
1148,mcarthur,2022-02-16T19:23:52.369000Z
